In [59]:
import torch
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name '_safe_tags' from 'sklearn.utils._tags' (C:\Users\josep\OneDrive\Desktop\Software\RedditProblemFinder\.venv\Lib\site-packages\sklearn\utils\_tags.py)

In [31]:
data = pd.read_csv('../data/labeled_cleaned_data.csv')

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModel.from_pretrained('bert-base-cased').to(device)

In [33]:
def get_bert_sentence_embedding(text):
    if pd.isna(text) or not isinstance(text, str) or text.strip() == "":
        return np.zeros(model.config.hidden_size, dtype=np.float32)

    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    sentence_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return sentence_embedding

In [34]:
data['title'] = data['title'].apply(lambda x: get_bert_sentence_embedding(x))
data['body'] = data['body'].apply(lambda x: get_bert_sentence_embedding(x))

AxisError: axis 1 is out of bounds for array of dimension 1

In [49]:
title_emb = np.stack(data.title.values)
body_emb = np.stack(data.body.values)
combined = np.concatenate([title_emb, body_emb], axis=1)

In [51]:
combined = pd.DataFrame(combined, columns=[f'bert_emb_{i}' for i in range(combined.shape[1])])
data_clean = data.drop(columns=['title', 'body'])  # drop embeddings stored in these two columns
data_combined = pd.concat([data_clean.reset_index(drop=True), combined], axis=1)

In [52]:
data_combined

,score,num_comments,upvote_ratio,created_utc,total_awards_received,title_length,body_length,title_word_count,body_word_count,has_question_mark,...,bert_emb_1526,bert_emb_1527,bert_emb_1528,bert_emb_1529,bert_emb_1530,bert_emb_1531,bert_emb_1532,bert_emb_1533,bert_emb_1534,bert_emb_1535
0,0,10,0.23,1.747252e+09,0,23,1089,5,190,1,...,0.327762,0.357278,-0.220949,-0.252389,-0.000575,0.018712,0.063588,-0.332531,0.145222,0.176384
1,2,1,1.00,1.747258e+09,0,81,1341,11,183,0,...,0.419177,0.331732,-0.333891,-0.228752,0.352128,0.289149,0.167564,-0.207953,0.273443,0.123073
2,2,0,1.00,1.747142e+09,0,97,5252,15,773,1,...,0.524374,0.305135,-0.026640,0.173032,0.124249,0.041688,0.049730,-0.416709,0.192840,0.117760
3,12,11,1.00,1.747235e+09,0,52,4172,10,656,0,...,0.573908,0.084166,-0.433981,0.296606,0.101162,0.226571,0.118238,-0.580175,0.092836,0.090041
4,1,3,1.00,1.746760e+09,0,77,3435,14,616,1,...,0.487628,0.455405,-0.368931,0.037845,0.190290,0.199070,0.157979,-0.535935,0.370099,0.298351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,2,1,1.00,1.747264e+09,0,36,673,7,96,1,...,0.449083,0.395922,-0.337725,-0.139512,0.258261,-0.041969,0.015981,-0.217825,0.524079,0.129853
494,1,0,1.00,1.747254e+09,0,44,1328,6,218,0,...,0.477509,0.172439,-0.174526,-0.018774,0.112075,0.122162,0.169146,-0.163527,0.422487,0.337666
495,1,0,1.00,1.747266e+09,0,31,468,5,86,1,...,0.248849,0.270873,-0.265309,-0.511417,-0.054680,-0.045479,0.046454,-0.281854,0.048485,0.133614
496,6,6,0.88,1.747252e+09,0,17,250,3,46,1,...,0.104349,0.194024,-0.233074,-0.148121,0.062550,0.142849,0.004142,-0.220661,0.132999,-0.114103


In [57]:


# Step 1: Prepare your data
X = data_combined.drop("market_viability", axis=1)  # Replace "target" if your column has a different name
y = data_combined["market_viability"]

neg = (y == 0).sum()
pos = (y == 1).sum()
scale_pos_weight = neg / pos

# Step 2: Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Apply SMOTE to training data only
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Step 3: Train the XGBoost classifier
xgb_model = XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    random_state=42
)
xgb_model.fit(X_resampled, y_resampled)

# Step 4: Evaluate
y_pred = xgb_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))


ImportError: cannot import name 'line_search_wolfe1' from 'sklearn.utils.fixes' (C:\Users\josep\OneDrive\Desktop\Software\RedditProblemFinder\.venv\Lib\site-packages\sklearn\utils\fixes.py)